In [ ]:
import scipy.stats

import polars as pl


annotations_df = (
    pl.scan_parquet("//allen/programs/mindscope/workgroups/dynamicrouting/ben/unit_drift.parquet")
    .select('unit_id', 'drift_rating')
    .filter(pl.col('unit_id').str.ends_with('_ks4').not_())
    .with_columns(
        session_id=pl.col('unit_id').str.split('_').list.slice(0, 2).list.join('_'),
    )
    .drop_nulls('drift_rating')
    # get spike-counts ------------------------------------------------- #
    .join(
        other=(
            pl.scan_parquet("//allen/programs/mindscope/workgroups/dynamicrouting/ben/spike_counts.parquet")
        ),
        on='unit_id',
        how='left',
    )
    .join(
        other=(
            pl.scan_parquet("s3://aind-scratch-data/dynamic-routing/cache/nwb_components/v0.0.261/consolidated/trials.parquet")
            .select('session_id', 'trial_index', 'block_index', 'context_name', 'start_time', 'stim_name')
        ),
        on=['session_id', 'trial_index',],
        how='inner',
    )
).collect()

In [ ]:
df

unit_id,drift_rating,session_id
str,i32,str
"""620263_2022-07-26_F-225""",0,"""620263_2022-07-26"""
"""620263_2022-07-26_F-343""",1,"""620263_2022-07-26"""
"""620263_2022-07-26_F-356""",5,"""620263_2022-07-26"""
"""620263_2022-07-26_F-659""",1,"""620263_2022-07-26"""
"""620263_2022-07-26_F-400""",5,"""620263_2022-07-26"""
…,…,…
"""742903_2024-10-24_B-268""",1,"""742903_2024-10-24"""
"""742903_2024-10-24_A-402""",0,"""742903_2024-10-24"""
"""742903_2024-10-24_B-146""",5,"""742903_2024-10-24"""


: 

In [ ]:
import polars as pl
import tqdm

all_trials_df = pl.read_parquet(f"s3://aind-scratch-data/dynamic-routing/cache/nwb_components/v0.0.261/consolidated/trials.parquet")
dfs = []
for (session_id, *_), session_df in tqdm.tqdm(all_trials_df.group_by('session_id'), total=all_trials_df.n_unique('session_id'), unit='sessions'):
    
    intervals_with_counts_df = insert_spike_counts_per_interval(
        trials_frame=pl.read_parquet(f"s3://aind-scratch-data/dynamic-routing/cache/nwb_components/v0.0.261/trials/{session_id}.parquet"),
        starts=(pl.col("stim_start_time") - 2, pl.col("stim_start_time"), ),
        ends=(pl.col("stim_start_time"), pl.col('stim_start_time') + 3, ),
        col_names=("baseline", 'response'),
        units_frame=pl.scan_parquet(f"s3://aind-scratch-data/dynamic-routing/cache/nwb_components/v0.0.261/units/{session_id}.parquet"),
    )
    dfs.append(intervals_with_counts_df)

df: pl.DataFrame = pl.concat(dfs)
df.select('unit_id', 'basline', 'response').write_parquet('//allen/programs/mindscope/workgroups/dynamicrouting/ben/spike_counts.parquet')

  2%|▏         | 4/222 [02:02<1:58:54, 32.73s/sessions]

In [5]:
len(result_df), len(max_min_df)

unit_id,start_time,stop_time,quiescent_start_time,quiescent_stop_time,stim_start_time,stim_stop_time,response_window_start_time,response_window_stop_time,task_control_response_time,response_time,reward_time,post_response_window_start_time,post_response_window_stop_time,stim_name,block_index,context_name,trial_index,trial_index_in_block,repeat_index,is_response,is_correct,is_incorrect,is_hit,is_false_alarm,is_correct_reject,is_miss,is_go,is_nogo,is_rewarded,is_noncontingent_reward,is_contingent_reward,is_reward_scheduled,is_aud_stim,is_vis_stim,is_catch,is_target,is_aud_target,is_vis_target,is_nontarget,is_aud_nontarget,is_vis_nontarget,is_vis_context,is_aud_context,is_context_switch,is_repeat,is_opto,session_idx,date,subject_id,session_id,id,baseline,response
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64,str,i64,f64,f64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,i64,date,i64,str,i64,i64,i64
"""741137_2024-10-08_B-180""",5698.17385,5703.69511,5698.17385,5699.67508,5699.68329,5700.18329,5699.75847,5700.67588,5700.5258,5700.51571,5700.54321,5700.706257,5703.725497,"""sound1""",5,"""aud""",529,75.0,0.0,true,true,false,true,false,false,false,true,false,true,false,true,false,true,false,false,true,true,false,false,false,false,false,true,false,false,false,0,2024-10-08,741137,"""741137_2024-10-08""",529,0,0
"""741137_2024-10-08_B-150""",2330.85339,2336.35804,2330.85339,2332.35464,2332.36248,2332.86248,2332.43807,2333.35548,null,null,null,2333.385823,2336.388343,"""sound1""",0,"""vis""",27,27.0,0.0,false,true,false,false,false,true,false,false,true,false,false,false,false,true,false,false,true,true,false,false,false,false,true,false,false,false,false,0,2024-10-08,741137,"""741137_2024-10-08""",27,74,121
"""741137_2024-10-08_F-236""",3555.1622,3560.66678,3555.1622,3556.66342,3556.6715,3557.1715,3556.74686,3557.6643,null,null,null,3557.694687,3560.697197,"""sound1""",2,"""vis""",210,28.0,0.0,false,true,false,false,false,true,false,false,true,false,false,false,false,true,false,false,true,true,false,false,false,false,true,false,false,false,false,0,2024-10-08,741137,"""741137_2024-10-08""",210,0,7
"""741137_2024-10-08_A-130""",4808.91237,4814.46695,4808.91237,4810.43036,4810.460733,4810.961123,4810.51365,4811.43108,4811.28098,4811.27212,4811.2984,4811.461513,4814.497393,"""vis1""",4,"""vis""",398,33.0,0.0,true,true,false,true,false,false,false,true,false,true,false,true,false,false,true,false,true,false,true,false,false,false,true,false,false,false,false,0,2024-10-08,741137,"""741137_2024-10-08""",398,10,3
"""741137_2024-10-08_E-325""",4061.3362,4066.85743,4061.3362,4062.83745,4062.84517,4063.34517,4062.92083,4063.83829,null,null,null,4063.868667,4066.887777,"""sound2""",3,"""aud""",286,13.0,0.0,false,true,false,false,false,true,false,false,true,false,false,false,false,true,false,false,false,false,false,true,true,false,false,true,false,false,false,0,2024-10-08,741137,"""741137_2024-10-08""",286,0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""741137_2024-10-08_E-25""",4442.75562,4448.26025,4442.75562,4444.25694,4444.26413,4444.76413,4444.3403,4445.25774,null,null,null,4445.288107,4448.290637,"""sound2""",3,"""aud""",343,70.0,0.0,false,true,false,false,false,true,false,false,true,false,false,false,false,true,false,false,false,false,false,true,true,false,false,true,false,false,false,0,2024-10-08,741137,"""741137_2024-10-08""",343,35,34
"""741137_2024-10-08_E-125""",4751.08059,4756.6185,4751.08059,4752.59849,4752.628857,4753.129267,4752.68188,4753.59933,null,null,null,4753.629697,4756.648897,"""vis2""",4,"""vis""",390,25.0,0.0,false,true,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,true,false,true,true,false,false,false,false,0,2024-10-08,741137,"""741137_2024-10-08""",390,0,0
"""741137_2024-10-08_B-205""",4967.54516,4

In [4]:
import math
import tqdm
import scipy.stats

class NullResult:
    statistic = None
    pvalue = None

null_result = NullResult()

def get_samples(unit_df: pl.DataFrame, interval: str) -> list[pl.Series] | None:
    if interval == 'trial' and 'trial' not in unit_df.columns:
        unit_df = unit_df.with_columns(trial=pl.col('baseline') + pl.col('response'))
    if unit_df[interval].sum() == 0:
        # cannot perform test if all samples are the same (zero spikes is a common case)
        return None
    if unit_df.n_unique('block_index') < 2:
        # for Templeton we chunk spike counts into 3 segments of time
        return list(s[interval] for s in unit_df.sort('trial_index').iter_slices(math.ceil(len(unit_df) / 3)))
    return unit_df.group_by('block_index').agg(pl.col(interval)).get_column(interval)

pvalue_method = None
pvalue_method = scipy.stats.PermutationMethod(n_resamples=1_000)
midrank = False
results = []

for (session_id, *_), session_df in tqdm.tqdm(annotations_df.group_by('session_id'), total=annotations_df.n_unique('session_id'), unit='sessions'):
    
    intervals_with_counts_df = insert_spike_counts_per_interval(
        trials_frame=pl.read_parquet(f"s3://aind-scratch-data/dynamic-routing/cache/nwb_components/v0.0.261/trials/{session_id}.parquet"),
        starts=(pl.col("stim_start_time") - 2, pl.col("stim_start_time"), ),
        ends=(pl.col("stim_start_time"), pl.col('stim_start_time') + 3, ),
        col_names=("baseline", 'response'),
        units_frame=pl.scan_parquet(f"s3://aind-scratch-data/dynamic-routing/cache/nwb_components/v0.0.261/units/{session_id}.parquet").filter(pl.col('unit_id').is_in(session_df['unit_id'])),
    )
    
    iterable = tuple(intervals_with_counts_df.drop_nulls(['baseline', 'response']).sort('unit_id').group_by('unit_id', 'context_name', maintain_order=True ))
    for (unit_id, context_name, *_), unit_df in tqdm.tqdm(iterable):
        result = dict(
            unit_id=unit_id,
            context_name=context_name,
        )
        for interval in ('baseline', 'response', 'trial'):
            samples = get_samples(unit_df, interval)
            if samples is None:
                stats = null_result
            else:
                stats = scipy.stats.anderson_ksamp(samples, midrank=midrank, method=pvalue_method)
            result[f'ad_stat_{interval}'] = stats.statistic
            result[f'ad_p_{interval}'] = stats.pvalue
        results.append(result)
    
result_df = pl.DataFrame(results)
max_min_df = (
    result_df
    .select(
        'unit_id',
        ad_stat_max_baseline=pl.col('ad_stat_baseline').max().over('unit_id'),
        ad_stat_max_response=pl.col('ad_stat_response').max().over('unit_id'),
        ad_stat_max_trial=pl.col('ad_stat_trial').max().over('unit_id'),
        ad_p_min_baseline=pl.col('ad_p_baseline').min().over('unit_id'),
        ad_p_min_response=pl.col('ad_p_response').min().over('unit_id'),
        ad_p_min_trial=pl.col('ad_p_trial').min().over('unit_id'),
    )
    .unique('unit_id')
)
max_min_df.write_parquet('//allen/programs/mindscope/workgroups/dynamicrouting/ben/ad_test.parquet')
max_min_df

  0%|          | 0/159 [00:00<?, ?sessions/s]

: 

In [85]:
pl.scan_parquet("s3://aind-scratch-data/dynamic-routing/cache/nwb_components/v0.0.261/consolidated/units.parquet").filter(pl.col('unit_id') == unit_id).select('obs_intervals').collect()

obs_intervals
list[list[f64]]
"[[14.975021, 4767.923937]]"


In [88]:
pl.scan_parquet("//allen/programs/mindscope/workgroups/dynamicrouting/ben/spike_counts.parquet").filter(pl.col('unit_id') == unit_id).collect().drop_nulls()

trial_index,unit_id,baseline,response
i64,str,i64,i64
0,"""636766_2023-01-23_C-168""",0,0
1,"""636766_2023-01-23_C-168""",0,0
2,"""636766_2023-01-23_C-168""",0,0
3,"""636766_2023-01-23_C-168""",0,1
4,"""636766_2023-01-23_C-168""",0,0
…,…,…,…
89,"""636766_2023-01-23_C-168""",0,0
90,"""636766_2023-01-23_C-168""",0,0
91,"""636766_2023-01-23_C-168""",0,0


In [80]:
interval

'response'

In [78]:
unit_df

unit_id,drift_rating,session_id,trial_index,baseline,response,block_index,context_name,start_time,stim_name
str,i32,str,i64,i64,i64,i64,str,f64,str
"""636766_2023-01-23_C-168""",5,"""636766_2023-01-23""",91,0,0,1,"""aud""",1564.68196,"""sound1"""
"""636766_2023-01-23_C-168""",5,"""636766_2023-01-23""",92,0,0,1,"""aud""",1570.26992,"""sound1"""
"""636766_2023-01-23_C-168""",5,"""636766_2023-01-23""",93,0,1,1,"""aud""",1577.64278,"""sound1"""


In [23]:
max_min_df.describe()

statistic,unit_id,ad_stat_max_baseline,ad_stat_max_response,ad_stat_max_trial,ad_p_min_baseline,ad_p_min_response,ad_p_min_trial
str,str,f64,f64,f64,f64,f64,f64
"""count""","""7290""",7249.0,7279.0,7284.0,7249.0,7279.0,7284.0
"""null_count""","""0""",41.0,11.0,6.0,41.0,11.0,6.0
"""mean""",null,22.887552,25.844388,30.347851,0.038323,0.027792,0.022533
"""std""",null,31.344237,32.790018,34.912484,0.078592,0.067013,0.06085
"""min""","""626791_2022-08-15_A-124""",-1.870859,-1.697056,-1.874194,0.001,0.000999,0.001
"""25%""",null,2.9653,4.094793,5.461257,0.001,0.001,0.001
"""50%""",null,10.020355,11.997118,15.868298,0.001,0.001,0.001
"""75%""",null,28.876063,34.20802,43.36421,0.016482,0.004861,0.00114
"""max""","""742903_2024-10-24_F-593""",200.927871,189.769959,189.991276,0.25,0.25,0.25


In [19]:
len(result_df), len(max_min_df)

(72722, 7290)

In [17]:
len(result_df), len(max_min_df)

(14547, 7290)

In [11]:
result_df.filter(pl.col('unit_id') == '741137_2024-10-09_E-223')

unit_id,context_name,stim_name,ad_stat_baseline,ad_stat_response,ad_stat_trial,ad_p_baseline,ad_p_response,ad_p_trial
str,str,str,f64,f64,f64,f64,f64,f64
"""741137_2024-10-09_E-223""","""aud""","""sound2""",0.639378,3.154024,0.906961,0.204525,0.013418,0.152958
"""741137_2024-10-09_E-223""","""aud""","""catch""",1.087416,2.064874,1.093305,0.125753,0.043587,0.124952
"""741137_2024-10-09_E-223""","""aud""","""vis2""",-0.9903,1.889781,-0.535186,0.25,0.052689,0.25
"""741137_2024-10-09_E-223""","""vis""","""catch""",1.814996,5.037134,4.209285,0.057136,0.001761,0.004296
"""741137_2024-10-09_E-223""","""vis""","""vis2""",8.953638,8.717206,14.320727,0.001,0.001,0.001
"""741137_2024-10-09_E-223""","""vis""","""sound1""",10.020753,7.594551,15.535587,0.001,0.001,0.001
"""741137_2024-10-09_E-223""","""vis""","""vis1""",2.599881,7.867481,5.783227,0.024428,0.001,0.001
"""741137_2024-10-09_E-223""","""aud""","""vis1""",1.599188,6.490163,7.933298,0.07219,0.001,0.001
"""741137_2024-10-09_E-223""","""aud""","""sound1""",0.836414,25.777405,24.543309,0.165132,0.001,0.001


In [13]:
max_min_df.filter(pl.col('unit_id') == '741137_2024-10-09_E-223')

unit_id,ad_stat_max_baseline,ad_stat_max_response,ad_stat_max_trial,ad_p_min_baseline,ad_p_min_response,ad_p_min_trial
str,f64,f64,f64,f64,f64,f64
"""741137_2024-10-09_E-223""",10.020753,25.777405,24.543309,0.001,0.001,0.001
